<a href="https://colab.research.google.com/github/KissTK322/ParasiticEgg_Classification_and_Detection/blob/main/yolov8_Parasite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path_img = "/content/drive/Shareddrives/Workspace/NKla/yolo_data/images/train/"
train_path_label = "/content/drive/Shareddrives/Workspace/NKla/yolo_data/labels/train/"
val_path_img = "/content/drive/Shareddrives/Workspace/NKla/yolo_data/images/val/"
val_path_label = "/content/drive/Shareddrives/Workspace/NKla/yolo_data/labels/val/"
test_path = "/content/drive/Shareddrives/Workspace/NKla/yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split("/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/data/") ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 11000 images---


  0%|          | 0/8800 [00:00<?, ?it/s]

------ Training data created with 80% split 8800 images -------


  0%|          | 0/2200 [00:00<?, ?it/s]

------ Testing data created with a total of 2200 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 41.1/166.8 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/training_results name=result1

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/training_results, name=result1, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=N

In [ ]:
!yolo task=detect mode=predict model=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/training_results/result14/weights/best.pt conf=0.55 source=/content/drive/Shareddrives/Workspace/NKla/test/data


Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25846129 parameters, 0 gradients

image 1/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0001.jpg: 480x640 1 Paragonimus spp, 66.2ms
image 2/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0002.jpg: 640x480 1 Opisthorchis viverrine, 65.1ms
image 3/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0003.jpg: 640x352 1 Hookworm egg, 71.7ms
image 4/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0004.jpg: 640x480 1 Paragonimus spp, 28.9ms
image 5/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0005.jpg: 640x480 1 Trichuris trichiura, 28.2ms
image 6/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0006.jpg: 608x640 1 Enterobius vermicularis, 65.7ms
image 7/2200 /content/drive/Shareddrives/Workspace/NKla/test/data/0007.jpg: 480x640 1 Paragonimus spp, 29.1ms
image 8/2200 /content/drive/Shareddrives/Workspace/NKla/t

In [ ]:
!cp -r /content/drive/Shareddrives/Workspace/NKla/runs/detect/predict /content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/output

In [ ]:
!yolo mode=export model=/content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/training_results/result14/weights/best.pt format=onnx

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 218 layers, 25846129 parameters, 0 gradients

PyTorch: starting from /content/drive/Shareddrives/Workspace/NKla/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/training_results/result14/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (49.6 MB)
requirements: Ultralytics requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 166.6 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR =====